# 14-02 云部署实战

Fly.io 和 Cloudflare Workers 部署。

## 1. Fly.io 部署

In [ ]:
// fly.toml 配置
/*
app = "myapp"
primary_region = "sin"

[build]
  dockerfile = "Dockerfile"

[env]
  PORT = "8080"

[[services]]
  internal_port = 8080
  protocol = "tcp"

  [[services.ports]]
    port = 80
    handlers = ["http"]

  [[services.ports]]
    port = 443
    handlers = ["tls", "http"]
*/

// 部署命令
// fly launch
// fly deploy
// fly logs

## 2. Cloudflare Workers

In [ ]:
// worker.ts
export interface Env {
  API_KEY: string;
  KV_NAMESPACE: KVNamespace;
}

export default {
  async fetch(request: Request, env: Env): Promise<Response> {
    const url = new URL(request.url);
    
    if (url.pathname === '/api/hello') {
      return new Response(JSON.stringify({ message: 'Hello!' }), {
        headers: { 'Content-Type': 'application/json' }
      });
    }
    
    // KV 存储
    if (url.pathname === '/api/cache') {
      await env.KV_NAMESPACE.put('key', 'value');
      const value = await env.KV_NAMESPACE.get('key');
      return new Response(value);
    }
    
    return new Response('Not found', { status: 404 });
  }
};

## 3. 环境变量管理

In [ ]:
// .env 文件（不提交到 Git）
/*
DATABASE_URL=postgres://...
REDIS_URL=redis://...
OPENAI_API_KEY=sk-...
*/

// 加载环境变量
import 'dotenv/config';

console.log(process.env.DATABASE_URL);

// 配置验证
import { z } from 'zod';

const envSchema = z.object({
  DATABASE_URL: z.string().url(),
  PORT: z.string().default('3000'),
  NODE_ENV: z.enum(['development', 'production']).default('development')
});

const env = envSchema.parse(process.env);

## 练习

1. 部署一个应用到 Fly.io
2. 创建 Cloudflare Worker
3. 配置 CI/CD 流水线